In [47]:
##### Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "ouptut/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [48]:
#Generate Cities List



In [49]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


635

In [50]:
#Perform API Calls


In [ ]:
# Starting URL for Weather Map API Call
#url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
        

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tuktoyaktuk
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | upernavik
Processing Record 4 of Set 1 | kloulklubed
Processing Record 5 of Set 1 | mount isa
Processing Record 6 of Set 1 | clarence town
Processing Record 7 of Set 1 | vao
Processing Record 8 of Set 1 | punta arenas
Processing Record 9 of Set 1 | sicamous
Processing Record 10 of Set 1 | busselton
Processing Record 11 of Set 1 | kapaa
Processing Record 12 of Set 1 | jamestown
Processing Record 13 of Set 1 | lavrentiya
Processing Record 14 of Set 1 | lompoc
Processing Record 15 of Set 1 | east london
Processing Record 16 of Set 1 | iqaluit
Processing Record 17 of Set 1 | esperance
Processing Record 18 of Set 1 | frankenberg
Processing Record 19 of Set 1 | behshahr
Processing Record 20 of Set 1 | cap malheureux
Processing Record 21 of Set 1 | port elizabeth
Processing Record 22 of Set 1 | sinnamary
Processing Record 

Processing Record 34 of Set 4 | barrow
Processing Record 35 of Set 4 | belyy yar
Processing Record 36 of Set 4 | bluff
Processing Record 37 of Set 4 | nuuk
Processing Record 38 of Set 4 | toyooka
Processing Record 39 of Set 4 | muli
Processing Record 40 of Set 4 | salalah
Processing Record 41 of Set 4 | thompson
Processing Record 42 of Set 4 | shingu
Processing Record 43 of Set 4 | port lincoln
Processing Record 44 of Set 4 | zvishavane
Processing Record 45 of Set 4 | lorengau
Processing Record 46 of Set 4 | yarada
Processing Record 47 of Set 4 | alta floresta
Processing Record 48 of Set 4 | danville
Processing Record 49 of Set 4 | hue
Processing Record 0 of Set 5 | calvia
Processing Record 1 of Set 5 | airai
Processing Record 2 of Set 5 | emerald
Processing Record 3 of Set 5 | malanje
Processing Record 4 of Set 5 | salina
Processing Record 5 of Set 5 | bengkulu


In [40]:
# Convert array of JSONs into Pandas DataFrame
city_data_pd = pd.DataFrame(city_data)

# Show Record Count
city_data_pd.count()

City          565
Lat           565
Lng           565
Max Temp      565
Humidity      565
Cloudiness    565
Wind Speed    565
Country       565
Date          565
dtype: int64

In [41]:
 # Display the City Data Frame
city_data_pd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hilo,19.7297,-155.0900,78.53,81,100,8.05,US,1651372644
1,kodiak,57.7900,-152.4072,40.51,87,100,25.32,US,1651372691
2,lavrentiya,65.5833,-171.0000,29.30,81,99,11.97,RU,1651372532
3,avarua,-21.2078,-159.7750,82.45,78,20,9.22,CK,1651372523
4,antalaha,-14.9003,50.2788,71.38,92,9,4.97,MG,1651372692


In [42]:
#Inspect the data and remove the cities where the humidity > 100%.


In [43]:
city_data_pd.describe() 

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,5.650000e+02
mean,20.917652,15.176678,60.690956,71.456637,57.915044,7.845487,1.651373e+09
std,33.101145,90.187146,18.640264,21.341374,39.380349,5.532605,1.830290e+02
min,-54.800000,-179.166700,-0.720000,2.000000,0.000000,0.000000,1.651372e+09
25%,-6.876700,-64.349900,48.090000,62.000000,17.000000,3.830000,1.651373e+09
50%,26.124700,18.423200,64.170000,76.000000,72.000000,6.800000,1.651373e+09
75%,49.135700,95.321400,75.330000,87.000000,99.000000,10.650000,1.651373e+09
max,78.218600,179.316700,97.200000,100.000000,100.000000,28.990000,1.651373e+09


In [44]:
#  Get the indices of cities that have humidity over 100%.
dirty_city_data = city_data_pd[(city_data_pd["Humidity"] > 100)].index
                                  
dirty_city_data

Int64Index([], dtype='int64')

In [45]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = city_data_pd.drop(dirty_city_data, inplace=False)
clean_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hilo,19.7297,-155.0900,78.53,81,100,8.05,US,1651372644
1,kodiak,57.7900,-152.4072,40.51,87,100,25.32,US,1651372691
2,lavrentiya,65.5833,-171.0000,29.30,81,99,11.97,RU,1651372532
3,avarua,-21.2078,-159.7750,82.45,78,20,9.22,CK,1651372523
4,antalaha,-14.9003,50.2788,71.38,92,9,4.97,MG,1651372692


In [46]:
# Extract relevant fields from the data frame
lats = clean_city_data["Lat"]
max_temps = clean_city_data["Max Temp"]
humidity = clean_city_data["Humidity"]
cloudiness = clean_city_data["Cloudiness"]
wind_speed = clean_city_data["Wind Speed"]

# Export the City_Data into a csv
clean_city_data.to_csv(output_data_file, index_label="City_ID")

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

In [ ]:
# Latitude vs. Temperature Plot


In [ ]:
# Build scatter plot for latitude vs. temperature
plt.scatter(lats, 
            max_temps,
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Max Temperature (%s)" % time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

In [ ]:
#Latitude vs. Humidity Plot


In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(lats, 
            humidity,
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (%s)" % time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

In [ ]:
#Latitude vs. Cloudiness Plot


In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(lats, 
            cloudiness,
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness (%s)" % time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

In [ ]:
# Latitude vs. Wind Speed Plot


In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(lats, 
            wind_speed,
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed (%s)" % time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

In [ ]:
#Linear Regression


In [ ]:
# Create a function to create Linear Regression plots
def plot_linear_regression(x_values, y_values, title, text_coordinates):
    
    # Run regresson on southern hemisphere
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    # Plot
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,text_coordinates,fontsize=15,color="red")
    plt.xlabel('Latitude')
    plt.ylabel(title)
    print(f"The r-value is: {rvalue**2}")
    plt.show()

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_hemi_df = city_data_pd.loc[(city_data_pd["Lat"] >= 0)]
southern_hemi_df = city_data_pd.loc[(city_data_pd["Lat"] < 0)]

In [ ]:
#Max Temp vs. Latitude Linear Regression


In [ ]:
# Linear regression on Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Max Temp"]
plot_linear_regression(x_values, y_values, 'Max Temp',(6,30))

In [ ]:
# Linear regression on Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Max Temp"]
plot_linear_regression(x_values, y_values, 'Max Temp', (-55, 90))

In [ ]:
# Humidity (%) vs. Latitude Linear Regression


In [ ]:
 # Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]
plot_linear_regression(x_values, y_values, 'Humidity',(40,10))

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Humidity"]
plot_linear_regression(x_values, y_values, 'Humidity', (-50, 20))

In [ ]:
#Cloudiness (%) vs. Latitude Linear Regression


In [ ]:
 # Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Cloudiness"]
plot_linear_regression(x_values, y_values, 'Cloudiness', (40,10))

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Cloudiness"]
plot_linear_regression(x_values, y_values, 'Cloudiness', (-30,30))